### Production Features Pipeline - CSV Version

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to a csv file. This is an alternative version of the pipeline that DOES NOT utilize the Hopsworks.ai Feature Store and is less dependent on other platforms.

2. It scrapes the upcoming games for today, and saves the blank records back into the csv file so that they can be accessed by the model for the prediction.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. For occasional use from local machines, this option may work fine, but you may need to setup a proxy server.

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.dashboard_processing import (
    NBADataProcessor,
)


DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 05
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&DateFrom=04/27/25&DateTo=05/04/25
0     1610612743
1     1610612746
2     1610612745
3     1610612744
4     1610612743
5     1610612746
6     1610612765
7     1610612752
8     1610612750
9     1610612747
10    1610612745
11    1610612744
12    1610612765
13    1610612752
14    1610612753
15    1610612738
16    1610612746
17    1610612743
18    1610612754
19    1610612749
20    1610612744
21    1610612748
22    1610612739
23    1610612745
24    1610612749
25    1610612747
26    1610612750
27    1610612752
28    1610612765
29    1610612738
30    1610612753
31    1610612754
dtype: object
HOME_TEAM_ID 0     1610612743
1     1610612744
2     1610612746
3     1610612765
4     1610612747
5     1610612745
6     1610612752
7     1610612738
8     1610612743
9     1610612754
10    1610612744
11    1610612748
12    1610612749
13    1610612750
14    1610612765
15    1610612753
Name: HOME_TEAM_ID, dtype: objec

**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Sat
Sun
[['1610612754', '1610612739'], ['1610612744', '1610612745']]
['42400201', '42400157']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    #exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    #exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:


df_old = pd.read_csv(DATAPATH / 'games.csv')

df_old


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30941,2025-04-28 00:00:00,42400104.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,83.0,36.000,60.000,...,17.0,45.0,NaN,138.0,54.700,88.000,46.800,26.0,51.0,0
30942,2025-04-27 00:00:00,42400134.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,103.0,41.600,82.400,...,25.0,33.0,NaN,129.0,60.200,84.600,46.200,36.0,41.0,0
30943,2025-04-27 00:00:00,42400164.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,116.0,42.200,78.800,...,19.0,49.0,NaN,113.0,45.000,88.000,40.400,23.0,41.0,1
30944,2025-04-27 00:00:00,42400124.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,93.0,42.900,82.400,...,18.0,54.0,NaN,94.0,37.400,61.100,45.500,19.0,38.0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [11]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)


# save the new games to the database
df_old.to_csv(DATAPATH / 'games.csv', index=False)

df_old

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30957,2025-04-28 00:00:00,42400104.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,83.0,36.000,60.000,...,17.0,45.0,NaN,138.0,54.700,88.000,46.800,26.0,51.0,0
30958,2025-04-27 00:00:00,42400134.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,103.0,41.600,82.400,...,25.0,33.0,NaN,129.0,60.200,84.600,46.200,36.0,41.0,0
30959,2025-04-27 00:00:00,42400164.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,116.0,42.200,78.800,...,19.0,49.0,NaN,113.0,45.000,88.000,40.400,23.0,41.0,1
30960,2025-04-27 00:00:00,42400124.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,93.0,42.900,82.400,...,18.0,54.0,NaN,94.0,37.400,61.100,45.500,19.0,38.0,0


**Add Today's Matchups for Feature Engineering**

In [12]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [13]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005.0,1610612748.0,1610612750.0,2021.0,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0.0,0,0.0
1,2022-03-12,22101006.0,1610612741.0,1610612739.0,2021.0,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1.0,0,1.0
2,2022-03-12,22101007.0,1610612759.0,1610612754.0,2021.0,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0.0,0,0.0
3,2022-03-12,22101008.0,1610612744.0,1610612749.0,2021.0,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1.0,0,1.0
4,2022-03-12,22101009.0,1610612743.0,1610612761.0,2021.0,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30897,2025-04-29 00:00:00,42400175.0,1610612743.0,1610612746.0,2024.0,131.0,55.800,78.300,51.500,29.0,38.0,115.0,52.400,64.300,37.900,29.0,38.0,1.0,1,1.0
30911,2025-05-02 00:00:00,42400156.0,1610612744.0,1610612745.0,2024.0,107.0,41.100,81.800,30.600,26.0,42.0,115.0,44.900,71.700,40.000,23.0,46.0,0.0,1,0.0
30930,2025-05-03 00:00:00,42400177.0,1610612743.0,1610612746.0,2024.0,120.0,52.200,71.400,37.900,24.0,46.0,101.0,47.600,68.200,29.600,30.0,36.0,1.0,1,1.0
30962,2025-05-04,42400201.0,1610612739,1610612754,2024.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,1,0.0


**Feature Engineering**

In [14]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

# save file
df_combined.to_csv(DATAPATH / 'games_engineered.csv', index=False)


df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300008,1610612765,1610612754,2003,87,0.392090,0.742188,0.333008,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27591,2025-05-01 00:00:00+00:00,42400176,1610612746,1610612743,2024,111,50.000000,86.687500,30.796875,23,...,1.517187,-0.587500,1.000000,-1.142857,-3.9,-5.800000,2.000000,-4.714286,-4.6,-3.200000
27592,2025-05-02 00:00:00+00:00,42400156,1610612744,1610612745,2024,107,41.093750,81.812500,30.593750,26,...,4.973438,1.261458,6.000000,3.714286,7.6,5.866667,-1.000000,-7.285714,-5.8,-5.066667
27593,2025-05-03 00:00:00+00:00,42400177,1610612743,1610612746,2024,120,52.187500,71.375000,37.906250,24,...,-1.848437,1.473958,1.333333,-0.571429,1.3,5.533333,2.666667,3.142857,4.8,3.533333
27594,2025-05-04 00:00:00+00:00,42400157,1610612745,1610612744,2024,0,0.000000,0.000000,0.000000,0,...,-1.642187,-1.434375,-4.666667,-2.857143,-6.2,-6.400000,0.333333,6.142857,5.3,5.466667


**Process Data for Convenient Dashboarding**

In [15]:
processor = NBADataProcessor()
exported_files = processor.export_data_for_dashboard()

print("\nData Processing Complete!")
print(f"Files exported for Dashboards:")
for key, value in exported_files.items():
    if value:
        print(f"- {key}: {value}")

2025-05-04 09:02:51,109 INFO: Initialized NBADataProcessor with data_path=data, model_path=models
2025-05-04 09:02:51,110 INFO: Exporting data for dashboards to data
2025-05-04 09:02:51,111 INFO: Preparing today's games data
2025-05-04 09:02:51,112 INFO: Loading data from data\games_engineered.csv
2025-05-04 09:02:51,748 INFO: Successfully loaded data: 27596 rows, 245 columns
2025-05-04 09:02:51,748 INFO: Filtering for season: 2024
2025-05-04 09:02:51,750 INFO: Processing data for prediction
2025-05-04 09:02:51,790 INFO: Making predictions
2025-05-04 09:02:51,790 WARNING: Model not loaded, loading now...
2025-05-04 09:02:51,790 INFO: Loading model from models\model.pkl


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

2025-05-04 09:02:52,114 INFO: Successfully loaded model
2025-05-04 09:02:52,114 INFO: Removing unused features
2025-05-04 09:02:52,396 INFO: Successfully made predictions for 2 games
2025-05-04 09:02:52,398 INFO: Prepared predictions for 2 games today
2025-05-04 09:02:52,400 INFO: Preparing processed games data
2025-05-04 09:02:52,401 INFO: Loading data from data\games_engineered.csv


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2025-05-04 09:02:53,071 INFO: Successfully loaded data: 27596 rows, 245 columns
2025-05-04 09:02:53,072 INFO: Filtering for season: 2024
2025-05-04 09:02:53,076 INFO: Processing data for prediction
2025-05-04 09:02:53,120 INFO: Making predictions
2025-05-04 09:02:53,120 INFO: Removing unused features
2025-05-04 09:02:53,267 INFO: Successfully made predictions for 591 games


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

2025-05-04 09:02:53,275 INFO: Calculating daily running accuracy metrics
2025-05-04 09:02:53,280 INFO: Calculating team-specific daily running accuracy
2025-05-04 09:02:53,335 INFO: Calculating home/away daily running accuracy
2025-05-04 09:02:53,379 INFO: Calculating overall daily running accuracy
2025-05-04 09:02:53,464 INFO: Adding running accuracy metrics to games dataframe


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2025-05-04 09:02:53,481 INFO: Calculating team-specific prediction accuracy (legacy method)
2025-05-04 09:02:53,515 INFO: Calculated team accuracy for 30 teams
2025-05-04 09:02:53,516 INFO: Processed 591 completed games with 362 correct predictions
2025-05-04 09:02:53,524 INFO: Filtering columns for dashboard
2025-05-04 09:02:53,528 INFO: Filtered dataframe from 257 to 22 columns
2025-05-04 09:02:53,533 INFO: Exported filtered games data to data\games_dashboard.csv
2025-05-04 09:02:53,534 INFO: Exporting running accuracy metrics
2025-05-04 09:02:53,620 INFO: Exported running accuracy metrics to data\running_accuracy_metrics.csv
2025-05-04 09:02:53,621 INFO: Exported season summary to data\season_summary_stats.csv

Data Processing Complete!
Files exported for Dashboards:
- all_games: games_dashboard.csv
- season_summary: season_summary_stats.csv
- running_accuracy: running_accuracy_metrics.csv
